In [173]:
## load the necessities

import pandas as pd
import numpy as np
import pickle

import time


In [119]:
## This code occasionally crashes my old, slow computer. 

df = pd.read_csv('data/statcast_dumps/statcast2019.csv').dropna(subset=['events'])
df
#all_data = df.copy()
#df = df.sample(n=50_000)

##So I'm creating a pickle to replace it.abs

#pickle_out = open('pickles/hr_clf_data_subset.pickle','wb')
#pickle.dump(df, pickle_out)

##And now I can load that data with

#pickle_in = open('pickles/hr_clf_data_subset.pickle','rb')
#df = pickle.load(pickle_in)

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,0,NaN,2019-03-21,NaN,NaN,NaN,Hunter Strickland,595777.0,519326.0,...,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,NaN,NaN
1,1,1,NaN,2019-03-21,NaN,NaN,NaN,Hunter Strickland,460026.0,519326.0,...,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,NaN,NaN
5,5,5,NaN,2019-03-21,NaN,NaN,NaN,Hunter Strickland,640461.0,519326.0,...,4.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,NaN,NaN
7,7,7,NaN,2019-03-21,NaN,NaN,NaN,Fernando Rodney,553882.0,407845.0,...,4.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,NaN,NaN
12,12,12,NaN,2019-03-21,NaN,NaN,NaN,Fernando Rodney,570267.0,407845.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735427,2954,2954,SL,2019-10-01,87.7,-3.558429,4.960761,Max Scherzer,460075.0,453286.0,...,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,Standard,Standard
735428,2955,2955,FF,2019-10-01,98.1,-3.275855,5.123839,Max Scherzer,669374.0,453286.0,...,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,Strategic,Standard
735432,2959,2959,FC,2019-10-01,93.0,-3.419024,5.001775,Max Scherzer,519058.0,453286.0,...,0.0,2.0,2.0,0.0,2.0,0.0,2.0,0.0,Standard,Standard
735438,2965,2965,FF,2019-10-01,98.3,-3.325589,5.119030,Max Scherzer,518735.0,453286.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Infield shift,Standard


In [51]:
#print(df['des'].unique())
for i in df.columns:
    print("'{}'',".format(i))

'Unnamed: 0'',
'index'',
'pitch_type'',
'game_date'',
'release_speed'',
'release_pos_x'',
'release_pos_z'',
'player_name'',
'batter'',
'pitcher'',
'events'',
'description'',
'spin_dir'',
'spin_rate_deprecated'',
'break_angle_deprecated'',
'break_length_deprecated'',
'zone'',
'des'',
'game_type'',
'stand'',
'p_throws'',
'home_team'',
'away_team'',
'type'',
'hit_location'',
'bb_type'',
'balls'',
'strikes'',
'game_year'',
'pfx_x'',
'pfx_z'',
'plate_x'',
'plate_z'',
'on_3b'',
'on_2b'',
'on_1b'',
'outs_when_up'',
'inning'',
'inning_topbot'',
'hc_x'',
'hc_y'',
'tfs_deprecated'',
'tfs_zulu_deprecated'',
'fielder_2'',
'umpire'',
'sv_id'',
'vx0'',
'vy0'',
'vz0'',
'ax'',
'ay'',
'az'',
'sz_top'',
'sz_bot'',
'hit_distance_sc'',
'launch_speed'',
'launch_angle'',
'effective_speed'',
'release_spin_rate'',
'release_extension'',
'game_pk'',
'pitcher.1'',
'fielder_2.1'',
'fielder_3'',
'fielder_4'',
'fielder_5'',
'fielder_6'',
'fielder_7'',
'fielder_8'',
'fielder_9'',
'release_pos_y'',
'estimated_ba_usin

In [52]:
#Checking for in the park HR.

df[df['events'] == 'home_run'].dropna(subset=['hit_location'])['des']

318774    Ian Desmond hits an inside-the-park home run (...
289952    Hunter Pence hits an inside-the-park home run ...
Name: des, dtype: object

In [53]:
## Make a list of batted ball data. I'm also including batter handedness and home team (which is a proxy for the park.)
batted_ball_data = [
    #des,
    #game_type,
    #'events',
    'stand',
    'p_throws',
    'home_team',
    #away_team,
    #type,
    #'hit_location,
    #'hc_x',
    #'hc_y',
    'launch_speed',
    'launch_angle',
    'field_angle'
]

In [120]:
## Add a categorical variable for home runs.
df['is_homer'] = df['events'].apply(lambda x: True if x == 'home_run' else False)
df['stand'] = df['stand'].apply(lambda x: True if x == 'R' else False)
df['p_throws'] = df['p_throws'].apply(lambda x: True if x == 'R' else False)


In [121]:
##hc_x and hc_y are unusual. An internet search suggests that they are coordinates for
##displying the hit location on a field map for MLB's app.
##That's obvious data leakage, but I do want the relative angle that these numbers can imply.

## Running the code below shows that they aren't scaled in the same way, both have a minimum of 0.

df[df['events'] == 'home_run'][['hc_x','hc_y']].dropna().sort_values(by='hc_x', ascending=False)

## Scale these
## I'm assuming that the max of hc_y is about the same actual distance as the max of hc_x. 
df['hc_x'] = df['hc_x'] / df['hc_x'].max()
df['hc_y'] = df['hc_y'] / df['hc_y'].max()
df['field_angle'] = np.arctan(df['hc_y']/df['hc_x'])

In [56]:
## Before going any further, I want to see how well a basic model
## can classify this data based on the most obvious features: launch angle and launch speed

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier

## Get small test and train splits to train a decision tree.
temp = df.dropna(subset=['launch_angle','launch_speed','is_homer'])
X_train, X_test, y_train, y_test = train_test_split(temp[['launch_angle','launch_speed']], 
                                                    temp['is_homer'], 
                                                    test_size=5_000, 
                                                    train_size=10_000,
                                                    random_state = 12
                                                    )

baseline_clf = DecisionTreeClassifier(max_depth = 8).fit(X_train,y_train)
dummy_clf = DummyClassifier().fit(X_train, y_train)
print(y_test.sum()/len(y_test))
print("Dummy score: {}".format(dummy_clf.score(X_test,y_test)))
print("Train: {}".format(baseline_clf.score(X_train,y_train)))
print("Test: {}".format(baseline_clf.score(X_test,y_test)))
print("Cofusion:\n{}".format(confusion_matrix(y_test,baseline_clf.predict(X_test))))


0.0512
Dummy score: 0.9064
Train: 0.9784
Test: 0.968
Cofusion:
[[4684   60]
 [ 100  156]]


#### That was surprisingly bad

I though a decision tree would do better than that, but it only correctly classifies 156 of 216 home runs, which is a pretty weak recall rate. Precision is even worse, with 256 classifications (about .6) 

In [135]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline



temp = df.dropna(subset =batted_ball_data)
temp = temp[batted_ball_data+['is_homer']]
X_train, X_test, y_train, y_test = train_test_split(temp[batted_ball_data], 
                                                    temp['is_homer'], 
                                                    test_size=5_000, 
                                                    train_size=10_000,
                                                    random_state = 12
                                                    )

col_transf = ColumnTransformer([
    ('one_hot',OneHotEncoder(sparse=False),['home_team'])
    ],
    remainder = 'passthrough'
)
tree_clf_pipe = Pipeline(
    [('column_transformer', col_transf),
    ('tree_clf', DecisionTreeClassifier())]
)

In [81]:
tree_clf_pipe.fit(X_train,y_train)

dummy_clf = DummyClassifier().fit(X_train, y_train)
print(y_test.sum()/len(y_test))
print("Dummy score: {}".format(dummy_clf.score(X_test,y_test)))
print("Train: {}".format(tree_clf_pipe.score(X_train,y_train)))
print("Test: {}".format(tree_clf_pipe.score(X_test,y_test)))
print("Cofusion:\n{}".format(confusion_matrix(y_test,tree_clf_pipe.predict(X_test))))

## Let's turn that into something quicker to re-use.
def performance_summary(model,X,y,X_dev,y_dev):
    
    dummy_clf = DummyClassifier().fit(X, y)
    print(y_dev.sum()/len(y_dev))
    print("Dummy score: {}".format(dummy_clf.score(X_dev,y_dev)))
    print("Train: {}".format(model.score(X,y)))
    print("Test: {}".format(model.score(X_dev,y_dev)))
    print("Train Cofusion:\n{}".format(confusion_matrix(y,model.predict(X))))
    print("Cofusion:\n{}".format(confusion_matrix(y_dev,model.predict(X_dev))))

0.0556
Dummy score: 0.899
Train: 1.0
Test: 0.9658
Cofusion:
[[4649   73]
 [  98  180]]


In [157]:
from sklearn.neural_network import MLPClassifier

mlp_6 = MLPClassifier(
    hidden_layer_sizes=(6),
    activation = 'relu',
    max_iter = 1000,
    solver = 'sgd'
    learning_rate_init = .03,
    alpha = 0
)

col_transf = ColumnTransformer([
    ('one_hot',OneHotEncoder(sparse=False),['home_team']),
    ('scaler',StandardScaler(),['launch_speed','launch_angle','field_angle'])
    ],
    remainder = 'passthrough'
)

mlp_6_pipe = Pipeline(
    [('column_transformer', col_transf),
    ('mlp_clf', mlp_6)]
)



SyntaxError: invalid syntax (<ipython-input-157-19f8e52c14e1>, line 8)

In [73]:
mlp_6_pipe.fit(X_train,y_train)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot',
                                                  OneHotEncoder(sparse=False),
                                                  ['home_team']),
                                                 ('scaler', StandardScaler(),
                                                  ['launch_speed',
                                                   'launch_angle',
                                                   'field_angle'])])),
                ('mlp_clf',
                 MLPClassifier(alpha=0, hidden_layer_sizes=6,
                               learning_rate_init=0.03, max_iter=1000))])

In [74]:
performance_summary(mlp_6_pipe,X_train,y_train,X_test,y_test)

0.0556
Dummy score: 0.895
Train: 0.9791510745215439
Test: 0.9752
Cofusion:
[[4676   46]
 [  78  200]]


## First MLP does about the same as a decision tree (but overfits less.)

Let's try giving it more data.

In [136]:
## We up the training data to the full subset.

X_train, X_test, y_train, y_test = train_test_split(temp[batted_ball_data], 
                                                    temp['is_homer'], 
                                                    test_size=10_000, 
                                                    ##train_size=10_000,
                                                    random_state = 12
                                                    )
X_test, X_final_test, y_test, y_final_test = train_test_split(X_test, y_test, test_size = 5000)##I know the trandional is X_dev and X_test but I didn't start out this way
mlp_6_pipe.fit(X_train,y_train)
performance_summary(mlp_6_pipe,X_train,y_train,X_test,y_test)

## Fun, no so fun side note: on my first pass with more data, there was a total failure: 
## it predicted the majority class. (Uhg.)
## I forgot to scale the data, which immediately eliminated the issue.


,stand,p_throws,home_team,launch_speed,launch_angle,field_angle
388651,True,True,SF,76.2,26.0,0.611815
647539,True,True,LAD,102.0,-3.0,1.004972
231292,True,True,SF,82.9,-21.0,1.037215
621458,False,True,STL,103.4,14.0,0.453784
275326,True,False,CLE,82.9,-21.0,1.025046
...,...,...,...,...,...,...
220006,True,False,CIN,86.4,37.0,0.474307
241604,True,True,NYM,89.2,23.0,0.734034
121153,False,True,STL,94.9,30.0,0.377005
616181,True,True,CIN,90.0,15.0,0.744305


### PSA

I ran the following with a subset of the data, assuming that the results on a large (~25,000) training set would translate to the full set. They did not. 

In [149]:
## Let's see if a bigger network can do more...

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=2) #saves time; let's find something that looks promising, then get more rigorous.

parameters = {'hidden_layer_sizes':[40,100,(60,20)]}
mlp_grid = GridSearchCV(mlp_6,param_grid=parameters,return_train_score=True, cv=k_fold)
mlp_grid.fit(col_transf.fit_transform(X_train),y_train)

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             estimator=MLPClassifier(alpha=0, hidden_layer_sizes=6,
                                     learning_rate_init=0.03, max_iter=1000),
             param_grid={'hidden_layer_sizes': [40, 100, (60, 20)]},
             return_train_score=True)

In [150]:
## A larger network slightly over fits but does better on the training set.
mlp_grid.cv_results_

{'mean_fit_time': array([12.81085801, 13.76236391, 23.18039608]),
 'std_fit_time': array([ 1.07606316,  0.21776295, 13.92780542]),
 'mean_score_time': array([0.08179283, 0.10742295, 0.09744596]),
 'std_score_time': array([0.00201464, 0.00162661, 0.00010395]),
 'param_hidden_layer_sizes': masked_array(data=[40, 100, (60, 20)],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'hidden_layer_sizes': 40},
  {'hidden_layer_sizes': 100},
  {'hidden_layer_sizes': (60, 20)}],
 'split0_test_score': array([0.96963135, 0.85178339, 0.96670892]),
 'split1_test_score': array([0.94677911, 0.96480696, 0.94677911]),
 'mean_test_score': array([0.95820523, 0.90829517, 0.95674401]),
 'std_test_score': array([0.01142612, 0.05651178, 0.00996491]),
 'rank_test_score': array([1, 3, 2], dtype=int32),
 'split0_train_score': array([0.970247  , 0.85206246, 0.9676062 ]),
 'split1_train_score': array([0.94635752, 0.96498363, 0.94635752]),
 'mean_train_score':

In [151]:
## Let's try a deeper network. See if we can really capture the training data.


parameters = {'hidden_layer_sizes':[(50,20),(80,30),(100,40)]}
mlp_grid = GridSearchCV(mlp_6,param_grid=parameters,return_train_score=True, cv=k_fold)
mlp_grid.fit(col_transf.fit_transform(X_train),y_train)

GridSearchCV(cv=KFold(n_splits=2, random_state=None, shuffle=False),
             estimator=MLPClassifier(alpha=0, hidden_layer_sizes=6,
                                     learning_rate_init=0.03, max_iter=1000),
             param_grid={'hidden_layer_sizes': [(50, 20), (80, 30), (100, 40)]},
             return_train_score=True)

In [152]:
mlp_grid.cv_results_

{'mean_fit_time': array([16.54689515, 21.94701886, 16.95467925]),
 'std_fit_time': array([9.20941985, 1.09892941, 7.84905624]),
 'mean_score_time': array([0.09131181, 0.1143589 , 0.13669407]),
 'std_score_time': array([0.00076663, 0.000144  , 0.00514138]),
 'param_hidden_layer_sizes': masked_array(data=[(50, 20), (80, 30), (100, 40)],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'hidden_layer_sizes': (50, 20)},
  {'hidden_layer_sizes': (80, 30)},
  {'hidden_layer_sizes': (100, 40)}],
 'split0_test_score': array([0.94635752, 0.97135664, 0.94635752]),
 'split1_test_score': array([0.94677911, 0.94677911, 0.91464939]),
 'mean_test_score': array([0.94656831, 0.95906787, 0.93050346]),
 'std_test_score': array([0.00021079, 0.01228877, 0.01585407]),
 'rank_test_score': array([2, 1, 3], dtype=int32),
 'split0_train_score': array([0.94677911, 0.97271175, 0.94677911]),
 'split1_train_score': array([0.94635752, 0.94635752, 0.91484455]),

In [166]:
## Why not?
# 
parameters = {'hidden_layer_sizes':[(80,30),(100,40),(200,50)],
            #'learning_rate':['invscaling'],
            'learning_rate_init':[.01,.03],
            'random_state':[5]}
k_fold.n_splits = 3 ## Let's make sure these are getting about the same results.
mlp_grid = GridSearchCV(mlp_6,param_grid=parameters,return_train_score=True, cv=k_fold)
mlp_grid.fit(col_transf.fit_transform(X_train),y_train)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
             estimator=MLPClassifier(alpha=0, hidden_layer_sizes=6,
                                     learning_rate_init=0.03, max_iter=1000),
             param_grid={'hidden_layer_sizes': [(80, 30), (100, 40), (200, 50)],
                         'learning_rate_init': [0.01, 0.03],
                         'random_state': [5]},
             return_train_score=True)

In [167]:
mlp_grid.cv_results_

{'mean_fit_time': array([34.2814366 , 41.40813033, 51.08162777, 32.4341588 , 71.04437359,
        61.21731393]),
 'std_fit_time': array([10.44313809, 17.6699185 , 24.88505709,  3.59838792, 23.61255725,
        26.48859119]),
 'mean_score_time': array([0.07689341, 0.07464798, 0.08761986, 0.08658218, 0.13836193,
        0.13369068]),
 'std_score_time': array([0.00051584, 0.00180158, 0.00238551, 0.00129058, 0.00291153,
        0.00052116]),
 'param_hidden_layer_sizes': masked_array(data=[(80, 30), (80, 30), (100, 40), (100, 40), (200, 50),
                    (200, 50)],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate_init': masked_array(data=[0.01, 0.03, 0.01, 0.03, 0.01, 0.03],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_random_state': masked_array(data=[5, 5, 5, 5, 5, 5],
              mask=[False, False, False, False, F

#### Let's go with 100,40 for the hidden layers.

If increasing the amount of data creates gains over the smaller networks, we should be able to capture that. Training time was about the same for 100, (100,20) and (100,40).



In [171]:
parameters = {'hidden_layer_sizes':[(200,80)],
            #'learning_rate':['invscaling'],
            'learning_rate_init':[.005],#,.01,.02],
            'random_state':[5]}
k_fold.n_splits = 3 ## Let's make sure these are getting about the same results.
mlp_grid = GridSearchCV(mlp_6,param_grid=parameters,return_train_score=True, cv=k_fold)

start = time.time()
mlp_grid.fit(col_transf.fit_transform(X_train),y_train)
stop = time.time()
print(stop-start)

NameError: name 'time' is not defined

In [169]:
mlp_grid.cv_results_

{'mean_fit_time': array([ 60.68581128, 134.45602242, 123.82021133]),
 'std_fit_time': array([46.8980901 , 32.20335154, 16.36133001]),
 'mean_score_time': array([0.18083231, 0.16009402, 0.16086682]),
 'std_score_time': array([0.01082454, 0.00083109, 0.00432355]),
 'param_hidden_layer_sizes': masked_array(data=[(200, 80), (200, 80), (200, 80)],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate_init': masked_array(data=[0.05, 0.01, 0.02],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_random_state': masked_array(data=[5, 5, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'hidden_layer_sizes': (200, 80),
   'learning_rate_init': 0.05,
   'random_state': 5},
  {'hidden_layer_sizes': (200, 80),
   'learning_rate_init': 0.01,
   'random_state': 5},
  {'hidden_layer_sizes': (200, 80),
   'learning_rate_init': 0.0

In [148]:
mlp_100_40 = MLPClassifier(
    hidden_layer_sizes=(100,40),
    activation = 'relu',
    max_iter = 1000,
    learning_rate_init = .03,
    alpha = 0
)

mlp_100_40_pipe = Pipeline(
    [('column_transformer', col_transf),
    ('mlp_clf', mlp_100_40)]
)

In [147]:
mlp_100_40_pipe.fit(X_train,y_train)
performance_summary(mlp_100_40_pipe,X_train,y_train,X_test,y_test)

0.0506
Dummy score: 0.9016
Train: 0.9465683124565373
Test: 0.9494
Train Cofusion:
[[107533      0]
 [  6070      0]]
Cofusion:
[[4747    0]
 [ 253    0]]
